In [72]:
import sqlite3
from sqlite3 import Error

In [73]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [74]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)



In [75]:
def Free_stuff(conn):
    """
    SELECT `Facilities` WHERE membercost = 0.0
    """
    cur = conn.cursor()
    
    query2 = """
        SELECT * FROM `Facilities` WHERE membercost = 0.0
        """
    cur.execute(query2)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [76]:
def Less_Revenue(conn):
    """
    Listing facilities with revenue less than $1000
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name, sum(case 
                WHEN b.memid = 0 then b.slots * f.guestcost
                ELSE b.slots * f.membercost
                END) AS revenue,
            sum(CASE b.memid
                WHEN memid != 0 
                THEN f.membercost * b.slots
                ELSE f.guestcost * b.slots 
                END) AS revenue
        FROM Bookings AS b
        INNER JOIN Facilities AS f
        ON b.facid = f.facid
        GROUP BY f.name
        HAVING sum(case 
            WHEN b.memid = 0 then b.slots * f.guestcost
            ELSE b.slots * f.membercost
            END) < 1000
        ORDER BY revenue;  
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


In [77]:
def Recommended_members(conn):
    """
    Who recommended Who
    """
    cur = conn.cursor()
    
    query1 = """
SELECT m.surname, m.firstname, m.recommendedby AS recomender_id, rec.surname AS recomender_surname, rec.firstname AS recomender_firstname
FROM Members AS m
LEFT JOIN Members AS rec ON m.recommendedby = rec.memid
WHERE m.recommendedby != 0
ORDER BY m.surname, m.firstname;

        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)



In [78]:
def Facility_usage(conn):
    """
    Facility usage by Members
    """
    cur = conn.cursor()
    
    query1 = """
SELECT b.facid, COUNT( b.memid ) AS mem_usage, f.name
FROM (

SELECT facid, memid
FROM Bookings
WHERE memid !=0
) AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [79]:
def Monthly_usage(conn):
    """
    Monthly Usage by members
    """
    cur = conn.cursor()
    
    query1 = """
SELECT b.months, COUNT( b.memid ) AS mem_usage
FROM (
SELECT strftime('%m', starttime) AS months, memid
FROM Bookings
WHERE memid !=0
) AS b
GROUP BY b.months;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [80]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
        print("\n \n3. Free facilities ")
        Free_stuff(conn)
        print("\n \n3. Less Revenue Facilities")
        Less_Revenue(conn)
        print("\n \n3. Recommended members")
        Recommended_members(conn)
        print("\n \n3. Facility Usage by Members")
        Facility_usage(conn)
        print("\n \n3. Monthly Usage by Members")
        Monthly_usage(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)

 
3. Free facilities 
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)

 
3. Less Revenue Facilities
('Table Tennis', 180, 3690)
('Snooker Table', 240, 4180)
('Pool Table', 270, 4140)

 
3. Recommended members
('Bader', 'Florence', '9', 'Stibbons', 'Ponder')
('Baker', 'Anne', '9', 'Stibbons', 'Ponder')
('Baker', 'Timothy', '13', 'Farrell', 'Jemima')
('Boothe', 'Tim', '3', 'Rownam', 'Tim')
('Butters', 'Gerald', '1', 'Smith', 'Darren')
('Coplin', 'Joan', '16', 'Baker', 'Timothy')
('Crumpet', 'Erica', '2', 'Smith', 'Trac